In [ ]:
# Import libraries
import os
import cv2
import random
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from graph import draw_confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
# Set directories
root_dir = os.getcwd()
data_dir = os.path.join(root_dir, 'mosquitoes-dataset')
test_dir = os.path.join(root_dir, 'test-dataset')

In [ ]:
tf.random.set_seed(1234)
random.seed(1234)
np.random.seed(1234)

In [ ]:
IMG_DIM = 224
IMG_SIZE = (IMG_DIM, IMG_DIM)
IMG_SHAPE = IMG_SIZE + (3,)

In [ ]:
# Load data
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(data_dir,
                                                               seed=1234,
                                                               validation_split=0.2,
                                                               subset='both',
                                                               batch_size=32,
                                                               image_size=IMG_SIZE,
                                                               )

In [ ]:
class_names = train_ds.class_names
class_num = len(class_names)

In [ ]:
# Data augmentation
data_augment = tf.keras.Sequential([tf.keras.layers.RandomFlip('horizontal_and_vertical'),
                                    tf.keras.layers.RandomRotation(0.3),
                                    tf.keras.layers.RandomContrast(0.3),
                                    ])

In [ ]:
# Image resize
image_resize = tf.keras.layers.Resizing(IMG_DIM, IMG_DIM)

In [ ]:
# Image normalization
preprocess_input = tf.keras.applications.resnet50.preprocess_input

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.resnet50.ResNet50(input_shape=IMG_SHAPE,
                                                     include_top=False,
                                                     weights='imagenet',
                                                     )

In [ ]:
base_model.trainable = False

In [ ]:
base_model.summary()

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(class_num, activation='softmax')

In [ ]:
inputs = tf.keras.Input(shape=IMG_SHAPE)
x = data_augment(inputs )
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
base_learning_rate = 1e-4
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'],
              )

In [ ]:
initial_epochs = 10

In [ ]:
history = model.fit(train_ds,
                    epochs=initial_epochs,
                    validation_data=val_ds,
                    )

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

In [ ]:
plt.figure(figsize=(10, 10))

plt.subplot(2, 1, 1)
plt.plot(loss, color='teal', label='loss')
plt.plot(val_loss, color='orange', label='val_loss')
plt.xticks(range(history.epoch[-1] + 1))
plt.legend(loc='upper left')
plt.title('Training and Validation Loss')

plt.subplot(2, 1, 2)
plt.plot(acc, color='teal', label='accuracy')
plt.plot(val_acc, color='orange', label='val_accuracy')
plt.xticks(range(history.epoch[-1] + 1))
plt.legend(loc='upper left')
plt.title('Training and Validation Accuracy')

plt.show()

In [ ]:
fine_tune_layers = 10

In [ ]:
for layer in base_model.layers[-fine_tune_layers:]:
    layer.trainable = True

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(
                  learning_rate=base_learning_rate/10),
              metrics=['accuracy'],
              )

In [ ]:
fine_tune_epochs = 10
total_epochs = initial_epochs + fine_tune_epochs

history_fine = model.fit(train_ds,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1]+1,
                         validation_data=val_ds,
                         )

In [ ]:
loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

In [ ]:
plt.figure(figsize=(10, 10))

plt.subplot(2, 1, 1)
plt.plot(loss, color='teal', label='loss')
plt.plot(val_loss, color='orange', label='val_loss')
plt.xticks(range(history_fine.epoch[-1]+1))
plt.ylim([0, 1])
plt.plot([initial_epochs-1, initial_epochs-1],
         plt.ylim(), color='brown', label='fine_tuning')
plt.legend(loc='upper left')
plt.title('Training and Validation Loss')

plt.subplot(2, 1, 2)
plt.plot(acc, color='teal', label='accuracy')
plt.plot(val_acc, color='orange', label='val_accuracy')
plt.xticks(range(history_fine.epoch[-1]+1))
plt.ylim([min(plt.ylim()), 1])
plt.plot([initial_epochs-1, initial_epochs-1],
         plt.ylim(), color='brown', label='fine_tuning')
plt.legend(loc='upper left')
plt.title('Training and Validation Accuracy')

plt.show()

In [ ]:
test_ds = tf.keras.utils.image_dataset_from_directory(test_dir,
                                                      seed=1234,
                                                      image_size=IMG_SIZE,
                                                      batch_size=16,
                                                      )

In [ ]:
image_batch, label_batch = test_ds.as_numpy_iterator().next()
yhat = model.predict_on_batch(image_batch)

yhat = tf.nn.softmax(yhat)

print('Predictions:\n', yhat.numpy())

label_yhat = np.empty(0, dtype=int)
for i in range(16):
    label_yhat= np.append(label_yhat, np.argmax(yhat[i]))

print('True labels:\n', label_batch)
print('Predicted labels:\n', label_yhat)

In [ ]:
cm = confusion_matrix(label_batch, label_yhat)
print(cm)

In [ ]:
draw_confusion_matrix(cm, class_names)

In [ ]:
""" cm_disp = ConfusionMatrixDisplay.from_predictions(y_true=label_batch,
                                                  y_pred=label_yhat,
                                                  display_labels=class_names,
                                                  cmap=plt.cm.gray_r,
                                                  ) """

In [ ]:
""" plt.figure(figsize=(20, 20))
for i in range(35):
    plt.subplot(10, 5, i + 1)
    plt.imshow(image_batch[i].astype('uint8'))
    plt.title(class_names[np.argmax(yhat[i])])
    plt.axis('off') """

In [ ]:
""" plt.figure(figsize=(20, 20))
for index, image in enumerate(os.listdir(test_dir)):
    img = cv2.imread(os.path.join(test_dir, image))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, IMG_SIZE, interpolation=cv2.INTER_LINEAR)
    x = np.expand_dims(img, 0)

    yhat = model.predict(x)
    yhat = tf.nn.softmax(yhat[0])
    yhat = np.array(yhat)
    
    plt.subplot(10, 5, index + 1)
    plt.imshow(img.astype('uint8'))
    plt.title(class_names[np.argmax(yhat)])
    plt.axis('off') """

In [ ]:
""" model.save(os.path.join(root_dir, 'models',
           'seeded_model.keras')) """